# Experiment (2) BoW TFIDF

Configurations

In [ ]:
import mlflow
import os

# Set DagsHub credentials (you'll need to set these environment variables)
os.environ["MLFLOW_TRACKING_USERNAME"] = (
    "yahiaehab10"  # Replace with your DagsHub username
)
os.environ["MLFLOW_TRACKING_PASSWORD"] = (
    "b1e53999e80aecdcf2b9a9e6d0e2fea6ae02fa4c"  # Replace with your DagsHub token
)

# Set the correct MLflow tracking URI for DagsHub
mlflow.set_tracking_uri(
    "https://dagshub.com/yahiaehab10/end-to-end-sentiment-analysis.mlflow"
)

# Set the experiment name
mlflow.set_experiment("BoW vs TFIDF")

2025/09/20 12:31:55 INFO mlflow.tracking.fluent: Experiment with name 'BoW vs TFIDF' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/ccfbe3442f4c4b2ba736edf60de93dbd', creation_time=1758360715467, experiment_id='1', last_update_time=1758360715467, lifecycle_stage='active', name='BoW vs TFIDF', tags={}>

## Experiment

In [7]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os

import mlflow.sklearn

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [6]:
df = pd.read_csv("processed_reddit_data.csv").dropna(subset=["clean_comment"])
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [9]:
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    # 1. Vectorization
    if vectorizer_type == "bow":
        vectorizer = CountVectorizer(
            max_features=vectorizer_max_features, ngram_range=ngram_range
        )
    else:
        vectorizer = TfidfVectorizer(
            max_features=vectorizer_max_features, ngram_range=ngram_range
        )

    # 2. Prepare data & split
    X = df["clean_comment"].values
    y = df["category"].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # 3. Fit model
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # 4.5. Define and train the model
    with mlflow.start_run():
        # Log model and parameters to MLflow
        mlflow.set_tag("mlflow.run_name", f"{vectorizer_name}_{ngram_range}_rf")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Description tag
        mlflow.set_tag("description", f"Random Forest with {vectorizer_name} and ngram {ngram_range}, max_features={vectorizer_max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_name)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Model hyperparameters & Log them
        n_estimators = 200
        max_depth = 15
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
        model.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log confusion matrix
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if label not in ["accuracy", "macro avg", "weighted avg"]:
                mlflow.log_metric(f"{label}_precision", metrics["precision"])
                mlflow.log_metric(f"{label}_recall", metrics["recall"])
                mlflow.log_metric(f"{label}_f1-score", metrics["f1-score"])

        # Log confusion matrix as artifact
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=np.unique(y), yticklabels=np.unique(y))
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()
        os.remove("confusion_matrix.png")

        # Log the model
        mlflow.sklearn.log_model(model, f"rf_model_{vectorizer_name}_{ngram_range}")


# 6. Run experiments with different configurations
ngrams_ranges = [(1, 1), (1, 2), (1, 3)]
max_features = 5000

for ngram_range in ngrams_ranges:
    run_experiment("bow", ngram_range, max_features, "BoW")
    run_experiment("tfidf", ngram_range, max_features, "TFIDF")

2025/09/20 13:06:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run hilarious-fish-161 at: https://dagshub.com/yahiaehab10/end-to-end-sentiment-analysis.mlflow/#/experiments/1/runs/f7639e2ac6dd494aadd4621a977333b4
🧪 View experiment at: https://dagshub.com/yahiaehab10/end-to-end-sentiment-analysis.mlflow/#/experiments/1


2025/09/20 13:07:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run upbeat-cow-762 at: https://dagshub.com/yahiaehab10/end-to-end-sentiment-analysis.mlflow/#/experiments/1/runs/aa1542218b9f4859a3556a7b9bd636b0
🧪 View experiment at: https://dagshub.com/yahiaehab10/end-to-end-sentiment-analysis.mlflow/#/experiments/1


2025/09/20 13:07:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run big-crab-292 at: https://dagshub.com/yahiaehab10/end-to-end-sentiment-analysis.mlflow/#/experiments/1/runs/6dca592bba8442aa892c2838f4885895
🧪 View experiment at: https://dagshub.com/yahiaehab10/end-to-end-sentiment-analysis.mlflow/#/experiments/1


2025/09/20 13:09:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run sincere-gull-642 at: https://dagshub.com/yahiaehab10/end-to-end-sentiment-analysis.mlflow/#/experiments/1/runs/3b28efd7e58f4a7eafbac62abdade8a3
🧪 View experiment at: https://dagshub.com/yahiaehab10/end-to-end-sentiment-analysis.mlflow/#/experiments/1


2025/09/20 13:10:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run righteous-fowl-786 at: https://dagshub.com/yahiaehab10/end-to-end-sentiment-analysis.mlflow/#/experiments/1/runs/07afef98aeb740b39ac8a029428da26e
🧪 View experiment at: https://dagshub.com/yahiaehab10/end-to-end-sentiment-analysis.mlflow/#/experiments/1
